In [193]:
import numpy as np
import pygame
import random
import math

BLUE  = (0,0,255)
WHITE = (255,255,255)
BLACK = (0,0,0)
RED   = (255,0,0)

coefficients         = {}
coefficients['eta0'] = 1.0000
coefficients['wght'] = 0.0196
coefficients['eta1'] = 3.9804
coefficients['eta2'] = 0.2157
coefficients['eta3'] = 3.2353
coefficients['eta4'] = 0.5098
params               = {}
params['migration_exists'] = True
params['migration_coeff']  = 0.15
params['number_of_iter']   = 100
params['use_saved_data']   = False

class swarm(): # CUSTOMIZABLE OBJECT
    def __init__(self, func, lowerbounds, upperbounds, number_of_particles=30, display=True):
        
        self.lowerbounds  = lowerbounds
        self.upperbounds  = upperbounds
        self.nop          = number_of_particles
        self.display      = display
        self.func         = func
        self.member       = {}
        
        if self.display:
            self.screen       = pygame.display.set_mode((1000, 1000))
            pygame.display.set_caption("Particle Swarm Optimization")
            self.screen.fill(WHITE)
                    
        for ii in range(self.nop):
            self.member[str(ii)] = {'center':(0,0)}
            self.member[str(ii)]['best_value'] = 10**32
            
            self.member[str(ii)]['position']      = {str(jj): np.random.random()*\
                                                    (self.upperbounds[jj]-self.lowerbounds[jj])+\
                                                     self.lowerbounds[jj] for jj in range(len(self.lowerbounds))}
            self.member[str(ii)]['best_position'] = self.member[str(ii)]['position']
            self.member[str(ii)]['velocity']      = {str(jj): np.random.random()\
                                                     for jj in range(len(self.lowerbounds))}
            self.member[str(ii)]['center'] = (int(np.round(self.member[str(ii)]['position']['0'])),
                                              int(np.round(self.member[str(ii)]['position']['1'])))
        self._evaluate_fitness()
        self._distance()
            
    def _evaluate_fitness(self):
        '''evaluates the fitness function for the members of the swarm'''
        self.fitness_values = {'_'           :{},
                               'sorted'      :{},
                               'best'        :{},
                               'best_sorted' :{}}
        for ii in range(self.nop):
            self.member[str(ii)]['fitness_value'] = self.func([self.member[str(ii)]\
                                                    ['position'][str(jj)] for jj in\
                                                    range(len(self.lowerbounds))])
            
            if self.member[str(ii)]['fitness_value'] < self.member[str(ii)]['best_value']:
                self.member[str(ii)]['best_value'] = self.member[str(ii)]['fitness_value']
                self.member[str(ii)]['best_position'] = self.member[str(ii)]['position']
        
            self.fitness_values['_'][str(ii)]    = self.member[str(ii)]['fitness_value']
            self.fitness_values['best'][str(ii)] = self.member[str(ii)]['best_value']

        self._sorted_fitness_value    = sorted(self.fitness_values['_'].items(), key = lambda kv:(kv[1], kv[0]))
        self.fitness_values['sorted'] = {str(self._sorted_fitness_value[ii][0]):self._sorted_fitness_value[ii][1]\
                                         for ii in range(self.nop)}
        
        self._sorted_best_value       = sorted(self.fitness_values['best'].items(), key = lambda kv:(kv[1], kv[0]))
        self.fitness_values['best_sorted'] = {str(self._sorted_best_value[ii][0]):self._sorted_best_value[ii][1]\
                                              for ii in range(self.nop)}
        
        self.best_position = {list(self.fitness_values['best_sorted'].keys())[ii]:\
                              self.member[list(self.fitness_values['best_sorted'].keys())[ii]]['best_position']\
                              for ii in range(self.nop)}
        
    def update(self): 
        '''updates the position and velocity of the members of the swarm'''
        for ii in range(self.nop):
            self.member[str(ii)]['delta']    = {str(jj): np.random.randint(-2,4) * np.random.random()\
                                                         for jj in range(len(self.lowerbounds))}
            
            self.member[str(ii)]['position'] = {str(jj): self.member[str(ii)]['position'][str(jj)] + \
                                                         self.member[str(ii)]['delta'][str(jj)]\
                                                         for jj in range(len(self.lowerbounds))}
            
            self.member[str(ii)]['center'] = (int(np.round(self.member[str(ii)]['position']['0'])),
                                              int(np.round(self.member[str(ii)]['position']['1'])))
            if self.display:
                pygame.draw.circle(self.screen,BLUE,self.member[str(ii)]['center'],2)
        
        self._evaluate_fitness()
        self._distance()
        
        if self.display:
            pygame.display.flip()     
            self.screen.fill(WHITE)
            
    def _distance(self):
        '''calculates the distances between the members of the swarm'''
        for ii in range(self.nop):
            self.member[str(ii)]['distance'] = {str(jj): {str(kk): self.member[str(ii)]['position'][str(kk)]-\
                                                         self.member[str(jj)]['position'][str(kk)]\
                                                         for kk in range(len(self.lowerbounds))}\
                                                         for jj in range(self.nop)}
            
            self.member[str(ii)]['distance_sorted'] = {str(jj):sorted(np.abs(self.member[str(ii)]['distance']\
                                                      [str(kk)][str(jj)]) for kk in range(self.nop)) for jj in\
                                                       range(len(self.lowerbounds))}
            
            self.member[str(ii)]['abs_distance'] = {str(jj): np.sqrt(math.fsum(self.member[str(ii)]['distance']\
                                                             [str(jj)][str(kk)]**2 for kk in\
                                                             range(len(self.lowerbounds)))) for jj in\
                                                             range(self.nop)}
            
            self.member[str(ii)]['abs_distance_sorted'] = [sorted(self.member[str(ii)]['abs_distance'].items(),\
                                                          key = lambda kv:(kv[1], kv[0]))[jj][1] for jj in\
                                                          range(self.nop)]
            
            self.member[str(ii)]['closest_neighbours'] = [sorted(self.member[str(ii)]['abs_distance'].items(),\
                                                          key = lambda kv:(kv[1], kv[0]))[jj][0] for jj in\
                                                          range(self.nop)]
            
            self.member[str(ii)]['closest_neighbours_fitness_values'] = {self.member[str(ii)]['closest_neighbours'][jj]:\
                                                                         self.member[self.member[str(ii)]['closest_neighbours'][jj]]['fitness_value']\
                                                                         for jj in range(self.nop)}
            
            self.member[str(ii)]['closest_neigbours_best'] = sorted(list(self.member[str(ii)]\
                                                                 ['closest_neighbours_fitness_values'].items())[1:5],
                                                                 key=lambda kv:(kv[1],kv[0]))[0][0]
            
            self.member[str(ii)]['distance_from_closest_best'] = self.member[str(ii)]['distance']\
                                                                [self.member[str(ii)]['closest_neigbours_best']]
    
    def optimize(self):
        '''PSO algoritm'''
        for ii in range(self.nop):
            for jj in range(self.lowerbounds):
                self.member[str(ii)]['velocity'][str(jj)] =  coefficients['eta0'] * (coefficients['wght'] * \
                                                             self.member[str(ii)]['velocity'][str(jj)]) + \
                                                             coefficients['eta1'] * np.random.random() * \
                                                             self.member[str(ii)]['best_position'][str(jj)] - \
                                                             self.member[str(ii)]['position'][str(jj)] + \
                                                             coefficients['eta2'] * np.random.random() * \
                                                             list(self.best_position.values())[0]['jj'] - \
                                                             self.member[str(ii)]['position'][str(jj)] + \
                                                             coefficients['eta3'] * np.random.random() * \
                                                             list(self.member[str[ii]]['distance_sorted'].values())[jj] + \
                                                             coefficients['eta4'] * np.random.random() * \
                                                             self.member[str(ii)]['distance_from_closest_best']
                
                                                      
                                                             
    


In [195]:
def egg(*args):
    x1    = args[0][0]
    x2    = args[0][1]
    term1 = -(x2+47) * math.sin(math.sqrt(math.fabs(x2+x1/2+47)))
    term2 = -x1 * math.sin(math.sqrt(math.fabs(x1-(x2+47))))
    out   = term1 + term2
    return out

def griewank(*args):
    d    = len(args[0])
    sum  = 0
    prod = 1
    for ii in range(d):
        xi   = xx[ii]
        sum  = sum + xi**2/4000
        prod = prod * math.cos(xi/math.sqrt(ii+1))
    out = sum - prod + 1
    return out

def bukin6(*args):
    x1 = args[0][0]
    x2 = args[0][1]    
    term1 = 100 * math.sqrt((abs(x2-0.01*x1**2)))
    term2 = 0.01 * abs(x1+10)    
    out = term1 + term2
    return out

def ackley(*args):
    d    = len(args[0])
    c    = math.pi * 2
    b    = 0.20
    a    = 20
    
    sum1 = 0
    sum2 = 0
    
    for ii in range(d):
        xi   = args[0][ii]
        sum1 = sum1 + xi**2
        sum2 = sum2 + math.cos(c*xi)
    
    term1 = -a * math.exp(-b*math.sqrt(sum1/d))
    term2 = -math.exp(sum2/d)
    
    y = term1 + term2 + a + math.exp(1)
    
    return y

def drop(*args):
    x1 = args[0][0]
    x2 = args[0][1]
    frac1 = 1+math.cos(12*math.sqrt(x1**2 + x2**2))
    frac2 = 0.5*(x1**2 + x2**2) + 2
    out = -frac1/frac2
    return out

In [196]:
fitness_function             = {}
fitness_function['egg']      = egg
fitness_function['griewank'] = griewank
fitness_function['bukin6']   = bukin6
fitness_function['ackley']   = ackley
fitness_function['drop']     = drop

In [197]:
particles = swarm(func=fitness_function['egg'],lowerbounds=[-512,-512],upperbounds=[512,512],
                  number_of_particles=50,display=True)

In [178]:
help(particles.update)

Help on method update in module __main__:

update() method of __main__.swarm instance
    updates the position and velocity of the members of the swarm



In [179]:
for ii in range(10):
    particles.update()

In [180]:
sorted(list(particles.member['39']['closest_neighbours_fitness_values'].items())[1:5],key=lambda kv:(kv[1],kv[0]))[0][0]

'18'

In [181]:
sorted(list(particles.member['39']['closest_neighbours_fitness_values'].values())[1:5])[0]

-282.14528276745153

In [182]:
particles.member['39']['closest_neigbours_best']

'18'

In [192]:
particles.member['39']['distance_from_closest_best']

{'0': -63.98989061765984, '1': -179.74105692434165}

In [191]:
particles.member['39']['distance'][particles.member['39']['closest_neigbours_best']]

{'0': -63.98989061765984, '1': -179.74105692434165}

In [189]:
{particles.member['39']['closest_neighbours'][ii]:particles.member[particles.member['39']['closest_neighbours'][ii]]['fitness_value'] for ii in range(particles.nop)}

{'39': -181.77124600091133,
 '21': -60.88616978272693,
 '20': -168.36991836064618,
 '13': 68.52357421761475,
 '18': -282.14528276745153,
 '24': -41.963549646538056,
 '31': 113.62350421445258,
 '27': -86.42840756334385,
 '3': -71.58064527484187,
 '12': 131.55513263788154,
 '44': 335.00263122139603,
 '26': -233.43301246046855,
 '9': 390.6135849533109,
 '34': -97.69255994364973,
 '37': -92.58187776596259,
 '47': -254.9311542068951,
 '30': 352.17242501979666,
 '41': 232.9736914163343,
 '35': 193.71025960305616,
 '32': 172.38901535927934,
 '5': 255.2630753210422,
 '25': -383.9814873746816,
 '40': 392.6892535008055,
 '49': -243.2684421596502,
 '15': -298.4046715988535,
 '11': -229.15542217032055,
 '42': -288.81364409087877,
 '36': -115.01098084596181,
 '48': 319.08706187540344,
 '33': 275.1855140078085,
 '10': 130.92212295313578,
 '2': -170.73345094604088,
 '4': -205.94346139423192,
 '28': -151.90104705055435,
 '19': -11.880628388838758,
 '1': -732.5558192455821,
 '23': 355.3024518903489,
 '

In [62]:
particles.member['3']['closest_neighbours']

['3',
 '28',
 '2',
 '8',
 '23',
 '25',
 '4',
 '9',
 '44',
 '27',
 '30',
 '34',
 '17',
 '20',
 '38',
 '46',
 '18',
 '0',
 '31',
 '48',
 '37',
 '13',
 '11',
 '24',
 '42',
 '10',
 '7',
 '16',
 '6',
 '35',
 '47',
 '12',
 '39',
 '32',
 '33',
 '22',
 '21',
 '15',
 '1',
 '19',
 '14',
 '41',
 '43',
 '45',
 '36',
 '40',
 '29',
 '5',
 '26',
 '49']

In [26]:
particles.member['3']['distance_sorted']

{'0': [0.0,
  12.376567463978688,
  14.06501685020072,
  14.764068243818713,
  17.441177587804532,
  28.8081128704477,
  39.462874422863365,
  44.42856847222674,
  45.81864349141051,
  52.456460605272696,
  57.36131882892727,
  68.68766969508687,
  79.61282831169086,
  89.6913205423852,
  90.46587546153404,
  118.43087676103443,
  169.839501684573,
  194.71265482955008,
  201.98345413527713,
  209.27470887656122,
  243.93906068685388,
  255.83033754094419,
  284.8155088202756,
  290.9094886532238,
  371.87417905475945,
  386.5190381567569,
  401.80725751724657,
  422.7006743108023,
  474.5519431142613,
  492.81162537976365,
  495.7778135031565,
  495.9333253076135,
  544.6845347131239,
  554.840000187469,
  559.6826798177266,
  599.401792956197,
  626.5129112309526,
  640.658420181898,
  645.4090664527363,
  659.9633744881818,
  689.3023304527238,
  724.9884369322642,
  728.6394070540824,
  742.3236417384135,
  783.8145771717568,
  807.4689672635543,
  837.6221235105917,
  872.23950883

In [10]:
particles.member['3']['closest_distances']

[0.0,
 72.55851551301328,
 144.8554255085327,
 192.97967954182695,
 221.60035784808528,
 272.7393139094239,
 274.099664645659,
 329.88254824244063,
 342.5887113234001,
 348.7420341941175,
 395.42009886392225,
 415.60549897876973,
 451.8890838815544,
 461.5049342753843,
 462.3113744253078,
 466.4051002314268,
 479.1785085708594,
 490.39971103021577,
 528.6940571800899,
 563.5288401324415,
 564.7494834751709,
 592.9657047691919,
 603.0998847897226,
 610.3244719721991,
 622.1535505872534,
 643.2434113479701,
 645.6860244743141,
 650.5382969927119,
 662.2281086742872,
 672.2829642040492,
 739.2648780765932,
 762.5999112638192,
 777.4499982952731,
 779.4018490380012,
 789.5637511387924,
 792.2530922231133,
 792.6428718491917,
 828.5448212363108,
 832.2378296591221,
 841.035168192145,
 841.5278122586176,
 897.8526585665318,
 923.6092534925989,
 949.1249822126116,
 957.4410279947818,
 963.7982544052722,
 989.0837950006637,
 1000.7809896459402,
 1061.2739987767584,
 1116.729946162912]

In [46]:
list(particles.best_position.values())[0]

{'0': 484.87244945526794, '1': 434.6804935772587}

In [12]:
particles.fitness_values['best_sorted']

{'33': -822.9153294344004,
 '32': -730.1830155158435,
 '2': -560.5330134758469,
 '27': -526.0956210763429,
 '40': -520.5582788711313,
 '25': -390.1233627442804,
 '38': -387.2005142883531,
 '16': -356.39436407257693,
 '39': -349.5122338381375,
 '46': -328.1289459569169,
 '41': -323.3277267532713,
 '42': -308.45125429414384,
 '14': -304.9606193343337,
 '6': -300.17425768322545,
 '9': -271.095991682922,
 '29': -240.64456059841646,
 '45': -224.9296260558165,
 '24': -224.19570082510108,
 '1': -195.36693184664105,
 '21': -188.6184756695307,
 '44': -180.0857524105291,
 '12': -164.64551123655886,
 '0': -152.61944490422266,
 '30': -132.79873431427512,
 '49': -83.81053866728395,
 '22': -60.06744224149955,
 '7': -37.82940231745809,
 '35': -27.41488600991739,
 '5': -26.594512031793556,
 '8': -23.86398929468855,
 '26': -21.1869477264657,
 '15': -11.096741775710868,
 '19': -5.650533143331526,
 '3': -1.4560745234020942,
 '28': 122.81913745757274,
 '10': 136.79998069317588,
 '34': 137.4860766269685,
 

In [13]:
list(particles.fitness_values['best_sorted'].values())

[-822.9153294344004,
 -730.1830155158435,
 -560.5330134758469,
 -526.0956210763429,
 -520.5582788711313,
 -390.1233627442804,
 -387.2005142883531,
 -356.39436407257693,
 -349.5122338381375,
 -328.1289459569169,
 -323.3277267532713,
 -308.45125429414384,
 -304.9606193343337,
 -300.17425768322545,
 -271.095991682922,
 -240.64456059841646,
 -224.9296260558165,
 -224.19570082510108,
 -195.36693184664105,
 -188.6184756695307,
 -180.0857524105291,
 -164.64551123655886,
 -152.61944490422266,
 -132.79873431427512,
 -83.81053866728395,
 -60.06744224149955,
 -37.82940231745809,
 -27.41488600991739,
 -26.594512031793556,
 -23.86398929468855,
 -21.1869477264657,
 -11.096741775710868,
 -5.650533143331526,
 -1.4560745234020942,
 122.81913745757274,
 136.79998069317588,
 137.4860766269685,
 138.71780199918675,
 141.46059091679672,
 146.20856816029436,
 158.34467619365688,
 172.33271257374827,
 206.36431861063815,
 257.29275780802755,
 258.5475040487983,
 378.33351745726554,
 400.5486448451458,
 496.0

In [14]:
{list(particles.fitness_values['best_sorted'].keys())[ii]:particles.member[list(particles.fitness_values['best_sorted'].keys())[ii]]['best_position'] for ii in range(particles.nop)}

{'33': {'0': 414.2446151759086, '1': 433.730213605653},
 '32': {'0': 375.71923864725727, '1': 504.97218460011527},
 '2': {'0': -309.5819036290191, '1': -488.63016784667457},
 '27': {'0': -101.47150102885394, '1': 437.4968841598861},
 '40': {'0': 236.95517310145942, '1': -489.92827695884023},
 '25': {'0': -504.53695884809514, '1': -252.75898675454454},
 '38': {'0': 384.45450047573667, '1': -87.45739826278907},
 '16': {'0': -246.25548272462726, '1': -444.8774628081278},
 '39': {'0': -484.11294594259914, '1': -15.239195766988267},
 '46': {'0': -451.2631878309997, '1': 114.20639283452608},
 '41': {'0': -412.3860187745261, '1': 121.84790404219208},
 '42': {'0': -233.66147211741702, '1': 240.31981106933372},
 '14': {'0': -322.18587833510713, '1': -66.58003820190885},
 '6': {'0': -300.36600788387403, '1': -47.30904561174328},
 '9': {'0': -494.34561017476733, '1': -270.56812423156356},
 '29': {'0': 300.7802765175768, '1': 28.471355364011536},
 '45': {'0': -257.9628548403124, '1': 26.1025001668

In [28]:
particles.member[list(particles.fitness_values['best_sorted'].keys())[0]]['best_position']

{'0': 484.87244945526794, '1': 434.6804935772587}

In [29]:
particles.member[list(particles.fitness_values['best_sorted'].keys())[0]]['best_value']

-945.1333448436433

In [30]:
particles.member['3']['distance_sorted']['0'][1]

12.376567463978688

In [31]:
particles.member['3']['distance_sorted']['1'][1]

7.402066129284492

In [32]:
sorted([particles.member['3']['distance'][str(ii)]['0'] for ii in range(particles.nop)])

[-57.36131882892727,
 -52.456460605272696,
 -39.462874422863365,
 -14.764068243818713,
 -14.06501685020072,
 -12.376567463978688,
 0.0,
 17.441177587804532,
 28.8081128704477,
 44.42856847222674,
 45.81864349141051,
 68.68766969508687,
 79.61282831169086,
 89.6913205423852,
 90.46587546153404,
 118.43087676103443,
 169.839501684573,
 194.71265482955008,
 201.98345413527713,
 209.27470887656122,
 243.93906068685388,
 255.83033754094419,
 284.8155088202756,
 290.9094886532238,
 371.87417905475945,
 386.5190381567569,
 401.80725751724657,
 422.7006743108023,
 474.5519431142613,
 492.81162537976365,
 495.7778135031565,
 495.9333253076135,
 544.6845347131239,
 554.840000187469,
 559.6826798177266,
 599.401792956197,
 626.5129112309526,
 640.658420181898,
 645.4090664527363,
 659.9633744881818,
 689.3023304527238,
 724.9884369322642,
 728.6394070540824,
 742.3236417384135,
 783.8145771717568,
 807.4689672635543,
 837.6221235105917,
 872.2395088399235,
 877.893279523332,
 936.3081851037832]

In [33]:
sorted([particles.member['3']['distance'][str(ii)]['1'] for ii in range(particles.nop)])

[-612.657029420871,
 -586.7536866264657,
 -576.4850992034496,
 -503.7497951804273,
 -498.68498399373004,
 -485.1065486263352,
 -454.0747146044331,
 -452.77040065009317,
 -436.19641175295726,
 -374.3811744891201,
 -366.14241624193585,
 -366.0115370553003,
 -364.9203628232723,
 -301.8277034730136,
 -300.01080398016285,
 -288.5501706967067,
 -274.9539740715775,
 -269.36164690950227,
 -254.14400538896723,
 -248.7197276691711,
 -238.90358033141888,
 -161.1452798247417,
 -146.87975878969084,
 -134.51938367280454,
 -124.53467676038537,
 -107.02301776216513,
 -100.56260881669994,
 -92.04757453677892,
 -76.8043760965462,
 -61.40838433767355,
 -47.05629358980167,
 -7.402066129284492,
 0.0,
 45.90350779729678,
 46.28524220572231,
 72.21040530190297,
 97.83649420792602,
 127.06011974416501,
 148.8549464759358,
 170.55434126686467,
 218.12312670207726,
 221.0154273330306,
 222.6676564302431,
 228.54888790785733,
 278.9508984598002,
 300.0816313365781,
 324.0267364226071,
 325.71404775728047,
 362.9

In [34]:
{str(jj):sorted(np.abs(particles.member['3']['distance'][str(ii)][str(jj)]) for ii in range(particles.nop)) for jj in range(len(particles.lowerbounds))}

{'0': [0.0,
  12.376567463978688,
  14.06501685020072,
  14.764068243818713,
  17.441177587804532,
  28.8081128704477,
  39.462874422863365,
  44.42856847222674,
  45.81864349141051,
  52.456460605272696,
  57.36131882892727,
  68.68766969508687,
  79.61282831169086,
  89.6913205423852,
  90.46587546153404,
  118.43087676103443,
  169.839501684573,
  194.71265482955008,
  201.98345413527713,
  209.27470887656122,
  243.93906068685388,
  255.83033754094419,
  284.8155088202756,
  290.9094886532238,
  371.87417905475945,
  386.5190381567569,
  401.80725751724657,
  422.7006743108023,
  474.5519431142613,
  492.81162537976365,
  495.7778135031565,
  495.9333253076135,
  544.6845347131239,
  554.840000187469,
  559.6826798177266,
  599.401792956197,
  626.5129112309526,
  640.658420181898,
  645.4090664527363,
  659.9633744881818,
  689.3023304527238,
  724.9884369322642,
  728.6394070540824,
  742.3236417384135,
  783.8145771717568,
  807.4689672635543,
  837.6221235105917,
  872.23950883

In [35]:
particles.fitness_values['_']

{'0': 493.4769140934799,
 '1': -99.4768491539817,
 '2': 389.7631304261574,
 '3': 250.04677085127372,
 '4': -78.8712858842456,
 '5': -371.5769918144683,
 '6': 114.11690502820687,
 '7': -55.94470536246227,
 '8': 254.18141421088646,
 '9': -675.0617548829598,
 '10': 140.04843455219137,
 '11': 379.9837677800679,
 '12': -165.04906394327435,
 '13': 689.5292528567118,
 '14': -305.77228426322495,
 '15': 246.6743708303788,
 '16': 224.78860765069882,
 '17': 47.384744576963456,
 '18': 279.03186287477257,
 '19': 182.53646251362682,
 '20': 610.382619491839,
 '21': -174.50570903993162,
 '22': 130.55616366406767,
 '23': 306.66610886064234,
 '24': 35.24830669473143,
 '25': 291.21256310328295,
 '26': -310.65405028213814,
 '27': -17.33478186531554,
 '28': -209.47723685558464,
 '29': 520.7229017344808,
 '30': -334.0401130924397,
 '31': -82.59607047698569,
 '32': 223.43517640171441,
 '33': 200.03534189995935,
 '34': 150.14768660050106,
 '35': -246.7920382045871,
 '36': -281.0855511449002,
 '37': -119.33193

In [36]:
particles.fitness_values['sorted']

{'47': -855.6895221093141,
 '43': -701.5336626456266,
 '9': -675.0617548829598,
 '49': -491.52818486691876,
 '5': -371.5769918144683,
 '48': -353.69607523202194,
 '30': -334.0401130924397,
 '41': -311.1430661358783,
 '26': -310.65405028213814,
 '14': -305.77228426322495,
 '36': -281.0855511449002,
 '35': -246.7920382045871,
 '28': -209.47723685558464,
 '42': -201.87775709525832,
 '21': -174.50570903993162,
 '12': -165.04906394327435,
 '37': -119.33193415928332,
 '40': -102.103733062293,
 '1': -99.4768491539817,
 '31': -82.59607047698569,
 '4': -78.8712858842456,
 '7': -55.94470536246227,
 '44': -19.364411946372428,
 '27': -17.33478186531554,
 '39': 2.6200867760424664,
 '45': 3.925865630475073,
 '38': 32.41585753049546,
 '24': 35.24830669473143,
 '17': 47.384744576963456,
 '6': 114.11690502820687,
 '22': 130.55616366406767,
 '10': 140.04843455219137,
 '34': 150.14768660050106,
 '19': 182.53646251362682,
 '33': 200.03534189995935,
 '32': 223.43517640171441,
 '16': 224.78860765069882,
 '1

In [37]:
particles.fitness_values['best']

{'0': 481.7438231003836,
 '1': -139.0600436923097,
 '2': 382.05865187376844,
 '3': 234.22856695339453,
 '4': -123.92706224374665,
 '5': -377.40155270022234,
 '6': 42.744645071836175,
 '7': -87.18925037616374,
 '8': 250.70870553942265,
 '9': -675.3301438540029,
 '10': 128.60922063302888,
 '11': 378.10472179620365,
 '12': -187.14722428245793,
 '13': 662.193127481594,
 '14': -322.69699738066134,
 '15': 218.63758418352205,
 '16': 171.21267712441264,
 '17': 9.483737917758958,
 '18': 279.03186287477257,
 '19': 144.57512614221338,
 '20': 570.8457364671524,
 '21': -174.50570903993162,
 '22': 46.47810205323373,
 '23': 297.8973152242074,
 '24': 22.626684510151016,
 '25': 286.4833165783755,
 '26': -360.80915212429494,
 '27': -100.74898383818186,
 '28': -226.8902746707648,
 '29': 520.7229017344808,
 '30': -338.1979839954727,
 '31': -83.51177501997054,
 '32': 213.21505066334697,
 '33': 197.7188139983829,
 '34': 115.34089071499928,
 '35': -246.7920382045871,
 '36': -330.27791732246266,
 '37': -138.1

In [38]:
particles.fitness_values['best_sorted']

{'47': -945.1333448436433,
 '43': -701.7118453195924,
 '9': -675.3301438540029,
 '49': -534.5079776451287,
 '5': -377.40155270022234,
 '48': -362.0401990931558,
 '26': -360.80915212429494,
 '30': -338.1979839954727,
 '36': -330.27791732246266,
 '14': -322.69699738066134,
 '41': -312.6100399182804,
 '42': -259.92088541556956,
 '35': -246.7920382045871,
 '28': -226.8902746707648,
 '12': -187.14722428245793,
 '21': -174.50570903993162,
 '1': -139.0600436923097,
 '37': -138.15629962360293,
 '4': -123.92706224374665,
 '40': -102.103733062293,
 '27': -100.74898383818186,
 '7': -87.18925037616374,
 '31': -83.51177501997054,
 '44': -32.31569123220876,
 '39': -22.202335092472637,
 '38': -3.131708872393645,
 '45': 3.4806728877236424,
 '17': 9.483737917758958,
 '24': 22.626684510151016,
 '6': 42.744645071836175,
 '22': 46.47810205323373,
 '34': 115.34089071499928,
 '10': 128.60922063302888,
 '19': 144.57512614221338,
 '16': 171.21267712441264,
 '33': 197.7188139983829,
 '32': 213.21505066334697,


In [43]:
list(particles.best_position.values())

[{'0': 484.87244945526794, '1': 434.6804935772587},
 {'0': -298.0467605692666, '1': -466.0062642756707},
 {'0': 401.7920810960434, '1': -358.12411396030836},
 {'0': -501.12022215855063, '1': -418.6517505163264},
 {'0': -446.31229770913643, '1': 112.92015006146488},
 {'0': 59.1573415338362, '1': -371.3590215908433},
 {'0': -260.67298808472304, '1': 476.2041269927536},
 {'0': 363.35143828329404, '1': 129.08696005252543},
 {'0': -409.7187734877587, '1': -185.1490974467314},
 {'0': -62.24295290742935, '1': 443.41117516769833},
 {'0': -349.23456162402323, '1': -93.81857841213467},
 {'0': 470.7340513399422, '1': 346.5670688759681},
 {'0': -113.10355289892834, '1': -310.84655778525484},
 {'0': 443.3867234772705, '1': -62.201057100239915},
 {'0': 26.064247436790396, '1': 317.3457464063919},
 {'0': -61.23064356556816, '1': 358.3927674753399},
 {'0': -216.6602387269831, '1': 228.47038612833273},
 {'0': 339.54724849745304, '1': 294.8968430332063},
 {'0': 228.92965987337266, '1': -239.953894731067

In [ ]:
particles.fitness_values['best_sorted'][list(particles.fitness_values['best_sorted'].keys())[0]]

In [ ]:
fitness_function['egg']([440.9767564295708,442.35882834525336])

In [ ]:
coefficients         = {}
coefficients['eta0'] = 1.0000
coefficients['wght'] = 0.0196
coefficients['eta1'] = 3.9804
coefficients['eta2'] = 0.2157
coefficients['eta3'] = 3.2353
coefficients['eta4'] = 0.5098
params               = {}
params['migration_exists'] = True
params['migration_coeff']  = 0.15
params['number_of_iter']   = 100
params['use_saved_data']   = False

In [ ]:
# Particle Swarm Optimization (PSO) Code%% Girdiler
minimization = 1             # 0: maximization, 1: minimization
coeff        = 1.0           # katsayı // 1.0
w            = 0.0196        # weight of the partical velocity  //w    = 0.0196;
eta1         = 3.9804        # eta1 = 3.9804 // partical best
eta2         = 0.2157        # eta2 = 0.2157   // global best
eta3         = 3.2353        # eta3 = 3.2353  // closest neighbour
eta4         = 0.5098        # eta4 = 0.5098 // closest best neighbour
eta5         = 0.00          # eta5 = 0.00 // average position of the swarm
eta6         = 0.00          # eta6 = 0.00 // average velocity of the swarm
#fitness_function = @egg;    # the function that will be optimized
number_of_particles = 150    # The number of the elements (particles/bees/ants/birds) inside the swarm
number_of_variables = 2      # variable sayısı / dimension sayısı
migration           = 1      # 0: no-migration, 1: migration
pmig                = 0.15   # migration probability if migration is allowed // 0.15
mig_katsayi         = 0.0110 # migration katsayısı // = pmig (0.15)
objective_number    = 1      # the number of the objectives that the fitness function gives out
iteration           = 200    # Number of iterations
saved_data          = 1      # 1: use saved x_encoded values, 0: start randomly

In [ ]:
lower_boundaries = zeros(1,number_of_variables);
upper_boundaries = zeros(1,number_of_variables);

lower_boundaries(1,:) = [-512 -512];
upper_boundaries(1,:) = [512 512];
% lower_boundaries(1,:) = 0 * ones(1,number_of_variables);
% upper_boundaries(1,:) = 100 * ones(1,number_of_variables);
vel_min = -20;
vel_max = 20;

position = zeros(number_of_particles,number_of_variables);
velocity = zeros(number_of_particles,number_of_variables);
[row col] = size(position);          %%% size of the domain row: number of particles, col: number of variables
globalbest_value(1,1:col) = 1000000;

distances = zeros(number_of_particles,number_of_particles);
distances(1:number_of_particles,1:number_of_particles*number_of_variables) = 0;
closest_distances(1:number_of_particles,1:4*number_of_variables) = 1000000000;     %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in mesafeler bulunuyor.
closest_particles(1:number_of_particles,1:4*number_of_variables) = 0;

%%%% Kaydedilmi� veri kullan�lmadan, rastgele bir ver iseti ile ba�lan�yor %%%%

if(saved_data == 0)
    %%
    %%% Her bir partical i�in rastgele pozisyon (x1, x2, ..., xn)  ve h�z
    %%% atamas� yap�l�yor.
    for i = 1:row
        for j = 1:col
            position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
            velocity(i,j) = random('unif',0,1);
        end
        fitness_values(i,:) = fitness_function(position(i,:));
    end
    personal_best_pos = position;
    
    total_velocity   = zeros(1,col);
    total_position   = zeros(1,col);
    average_velocity = zeros(1,col);
    average_position = zeros(1,col);
    for j = 1:col
        for i = 1:row
            total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
            total_position(1,j) = total_position(1,j) + position(i,j);
        end
        average_velocity(1,j) = total_velocity(1,j) / row;
        average_position(1,j) = total_position(1,j) / row;
    end
    
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%%%% Global en iyi fitness de�eri bulunuyor
    if(minimization == 1)
        for ii=1:row
            if (partical_best(ii,:) < globalbest_value)
                globalbest_value = partical_best(ii,:);
                global_best_index = ii;
                global_best_position = position(global_best_index,:);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    genn = 1;
    %%
    %%% kaydedilmi� veri kullan�l�yor
else
    load('PSO_saved_data.mat');
    [saved_data_birey_sayisi saved_data_col] = size(fitness_values);
    genn = genn_saved;
    if(row>saved_data_birey_sayisi)
        for i = saved_data_birey_sayisi+1:row
            for j = 1:col
                position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                velocity(i,j) = random('unif',0,1);
            end
            fitness_values(i,:) = fitness_function(position(i,:));
            personal_best_pos(i,:) = position(i,:);
        end
        %%
        %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
        for j = 1:col
            for i = 1:row
                for k = 1:row
                    column = (j-1)*row + k;
                    distances(i,column) = abs(position(i,j) - position(k,j));
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
        %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
            end
        end
        
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sutun = (j-1) * 4;
                closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
            end
        end
        for j = 1:col
            for i=1:row
                for k=1:4
                    column = (j-1)*row;
                    sutun = (j-1) * 4 + k;
                    closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
                end
            end
        end
        
        %%
        %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
        %%% de�eri kaydediliyor.
        for ii=saved_data_birey_sayisi+1:row
            partical_best(ii,:) = fitness_function(position(ii,:));
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                for k=1:4
                    sutun = (j-1) * 4 + k;
                    closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
        %%%%% en alttan en yukar�ya do�ru s�ralan�yor
        
        %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
        closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
            end
        end
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%% En yak�n particle'lar�n indisleri s�ralan�yor
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%%%% Global en iyi fitness de�eri bulunuyor
        if(minimization == 1)
            for ii=1:row
                if (partical_best(ii,:) < globalbest_value)
                    globalbest_value = partical_best(ii,:);
                    global_best_index = ii;
                    global_best_position = position(global_best_index,:);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    end
end

scatterplot(position(:,1:2));

iter = 0;

%%
%%%%%%% iterasyonlar ba�l�yor
while(iter < iteration)
    remainder = rem(iter,10);
    %%
    %%% Pozisyon ve h�z de�erleri g�ncelleniyor.
    for ii=1:row
        
        for jj=1:col
            velocity(ii,jj) = coeff * (w * velocity(ii,jj) ...
                + eta1 * random('unif',0,1) * (personal_best_pos(ii,jj) - position(ii,jj))...
                + eta2 * random('unif',0,1) * (global_best_position(:,jj) - position(ii,jj)))...
                + eta3 * random('unif',0,1) * (position(closest_particles(ii,(jj-1)*4+2),jj) - position(ii,jj))...
                + eta4 * random('unif',0,1) * (position(closest_best_particles_indices(ii,jj) ,jj) - position(ii,jj))...
                + eta5 * random('unif',0,1) * (average_position(1,jj) - position(ii,jj))...
                + eta6 * random('unif',0,1) * average_velocity(1,jj);
            
            if(velocity(ii,jj)>vel_max)
                velocity(ii,jj) = vel_max;
            elseif(velocity(ii,jj)<vel_min)
                velocity(ii,jj) = vel_min;
            end
            
            position(ii,jj) = position(ii,jj) + velocity(ii,jj);
            
            if(position(ii,jj)>upper_boundaries(1,jj))
                position(ii,jj) = upper_boundaries(1,jj);
            elseif(position(ii,jj)<lower_boundaries(1,jj))
                position(ii,jj) = lower_boundaries(1,jj);
            end
        end
        
        fitness_values(ii,:) = fitness_function(position(ii,:));
        
        if(fitness_values(ii,:) < partical_best(ii,:))
            partical_best(ii,:) = fitness_values(ii,:);
            personal_best_pos(ii,:) = position(ii,:);
        end
        
        if (partical_best(ii,:) < globalbest_value)
            globalbest_value = partical_best(ii,:);
            global_best_index = ii;
            global_best_position = position(global_best_index,:);
        end
        
        total_velocity   = zeros(1,col);
        total_position   = zeros(1,col);
        average_velocity = zeros(1,col);
        average_position = zeros(1,col);
        for j = 1:col
            for i = 1:row
                total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
                total_position(1,j) = total_position(1,j) + position(i,j);
            end
            average_velocity(1,j) = total_velocity(1,j) / row;
            average_position(1,j) = total_position(1,j) / row;
        end
    end
    %%
    %%%% Migration eklemesi yap�l�yor e�er migration=1 ise %%%
    if(migration==1)
        migration_probability = random('unif',0,1);
        total_migrating_particle_number = round(pmig*number_of_particles+objective_number);
        migrating_particle_number1 = round((pmig*number_of_particles+objective_number)/4);
        migrating_particle_number2 = 2 * migrating_particle_number1;
        migrating_particle_number3 = 3 * migrating_particle_number1;
        if(migration_probability <= pmig)
            sorted = sort(fitness_values(:,objective_number),'descend');
            
            %%% Fitness de�eri en d���k olan particle'lar�n indeksleri
            %%% bulunuyor
            for wi=1:total_migrating_particle_number
                worst_indeks(wi,1) = find(fitness_values(:,objective_number)==sorted(wi,1),1,'first');
            end
            
            %%% Fitness de�eri en y�ksek olan particle'lar�n indeksleri
            %%% bulunuyor
            for bi=1:(0.20*number_of_particles)
                best_indeks(bi,1) = find(fitness_values(:,objective_number)==sorted(number_of_particles+1-bi,1),1,'first');
                best_particle_array(bi,:) = position(best_indeks(bi,1),:);
            end
            
            %%% Fitness de�eri en y�ksek particle'lar�n average
            %%% pozisyonlar� bulunuyor.
            for j=1:col
                best_particles_average_position(1,j) = averaging(best_particle_array(:,j));
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n yar�s�, random
            %%% particle'lar ile de�i�tiriliyor
            for i = 1:migrating_particle_number1
                for j = 1:col
                    position(worst_indeks(i),j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1);
                end
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n kalan yar�s�, en
            %%% iyi particle'lar�n ortalama pozisyon ve h�z de�eri merkez
            %%% al�narak random pozisyon ve h�z de�erlerine sahip yeni
            %%% particle'lar ile de�i�tiriliyor.
            for i = migrating_particle_number1+1:migrating_particle_number2
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 0.10 * (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number2+1:migrating_particle_number3
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 10 * (best_particle_array(1,j) - best_particle_array(end,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number3+1:total_migrating_particle_number
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 2.0 * (best_particles_average_position(1,j) - average_position(1,j)) * random('unif',0,1) + sign * global_best_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            
        end
        clear worst_indeks;
    end
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%
    
    if (remainder == 1)
        scatterplot(position(:,1:2));
    end
    
    best_value_array(genn,1) = globalbest_value;
    
    genn = genn + 1;
    iter = iter + 1;
    text1 = 'iter = %d\n';
    fprintf(text1,iter);
    
end
genn_saved = genn;
save('PSO_saved_data.mat','position','velocity','closest_distances','closest_particles','globalbest_value','global_best_position','partical_best','personal_best_pos','closest_best_particles_indices','average_velocity','average_position','fitness_values','genn_saved','best_value_array');
close all;
scatterplot(position(:,1:2));